In [1]:
def scrap_football_data (url): 
     import requests 
     import re 
     import pandas as pd 
 
     r = requests.get(url) 
     html_code = r.text   

     ## Find all teams in source HTML
     hits = re.finditer("\"_BLANK\"><?B?>?[A-Z]{1}[a-z]?\.?[A-Z]?\.?\s?[a-z]*\s?[A-Z]?[a-z]*<?/?B?>?</A>", html_code)
     teams = [m.group(0) for m in hits]

     ## Removing tie breaker
     for i in range(0,len(teams)):
          if((i != 0 and i !=1) and i < (len(teams)-1)) and (teams[i] == teams[i-2] and teams[i+1] == teams[i-1]) :
               teams.pop(i+1)
               teams.pop(i)

     ## Sorting into Home, Away, and Winning Team lists
     Home_Team = []
     Away_Team = []
     Winner_Team = []

     for i in range(len(teams)):
          name = re.finditer(">[A-Z]{1}[a-z]?\.?[A-Z]?\.?\s?[a-z]*\s?[A-Z]?[a-z]*<",teams[i])
          teamName = [m.group(0) for m in name]
          teamNameClean = teamName[0][1:len(teamName[0])-1]
          if (i % 2) == 0:
                Home_Team.append(teamNameClean)
          else:
               Away_Team.append(teamNameClean)
          if re.search("<B>",teams[i]) != None:
                Winner_Team.append(teamNameClean) 


     #Find predictions in source HTML
     predHits = re.finditer("<NOBR><?B?>?[0-9]{1,2}<?/?B?>?%</NOBR>", html_code)
     preds = [m.group(0) for m in predHits]       

     ## Sort and Clean up predictions   
     Prediction_Home_Team = []
     Prediction_Away_Team = []

     for i in range(len(preds)):
          num = re.finditer("[0-9]{1,2}",preds[i])
          prediction = [m.group(0) for m in num]
          predClean = prediction[0] + "%"
          if (i % 2) == 0:
               Prediction_Home_Team.append(predClean)
          else:
               Prediction_Away_Team.append(predClean)  

     results = pd.DataFrame(list(zip(Home_Team, Away_Team, Winner_Team, Prediction_Home_Team, Prediction_Away_Team)),
columns=['Home', 'Away', 'Winner', 'Home Team Prediction', 'Away Team Prediction'])

     return results

In [2]:
scrap_football_data("https://probabilityfootball.com/picks.html?1487349677&username=AVERAGES&weeknum=5")

,Home,Away,Winner,Home Team Prediction,Away Team Prediction
0,New Orleans,Indianapolis,Indianapolis,31%,69%
1,Atlanta,Minnesota,Minnesota,39%,61%
2,Carolina,St. Louis,Carolina,45%,55%
3,Denver,Buffalo,Denver,60%,40%
4,Kansas City,Houston,Houston,48%,52%
5,Miami,Washington,Washington,45%,55%
6,New England,N.Y. Jets,New England,71%,29%
7,Philadelphia,Green Bay,Green Bay,59%,41%
8,Pittsburgh,Cleveland,Pittsburgh,67%,33%
9,Tennessee,Jacksonville,Tennessee,39%,61%


In [3]:
import pandas as pd
url = "https://probabilityfootball.com/picks.html?1487349677&username=AVERAGES&weeknum="
allSeason2008List = []
for i in range(5,22):
    tempDF = scrap_football_data(url+ str(i))
    allSeason2008List.append(tempDF)

final_DF = pd.concat(allSeason2008List, ignore_index=True)




In [4]:
predictions_winner = []
for i in range(len(final_DF)):
   if final_DF.iloc[i]['Home'] == final_DF.iloc[i]['Winner']:
      predictions_winner.append(int(final_DF.iloc[i]['Home Team Prediction'][:-1]))
   else: predictions_winner.append(int(final_DF.iloc[i]['Away Team Prediction'][:-1]))
   ## IF winner == home, prediction_winner[i] = home_pred, else pred_winner[i] = away_pred

### One Sample T-Test
Null Hypothesis: The experts prediction accuracy is equal to 50%
Alternative Hypothesis: The experts prediction accuracy is greater than 50%

In [5]:
from scipy import stats
stats.ttest_1samp(a = predictions_winner, popmean = 50, alternative = "greater")


Ttest_1sampResult(statistic=7.354975028437655, pvalue=1.3033908235719028e-12)

### T-Test Results
Because the p-value is less than 0.05, we can reject the null hypothesis that the experts mean prediction accuracy is equal to 50%. Therefore, the experts mean prediction accuracy is greater than 50%, which was the alternative hypothesis. I would recommend that the group uses the expert driven forecasts. Since an unbiased coin has correctly predicts the winner of the NFL game 50% of the time, I do think it was an appropriate model to compare our model to because it shows that the expert driven model predicts the winners more accurately than an unbiased coin.
